## 爬取目标

<div align=center>
    <img src="../../../images/fb0cc800f904f30931b46f0af6dd6c257e0a709d67c0dec84766a0f3562b9be4.png" width=75%>
    </img>
</div>

- 1. 提取每一页的电影列表,以及每个电影的详情页

- 2. 提取每部电影的`名称`、`封面`、`类别`、`上映时间`、`评分`、`剧情简介`

- 3. 存入数据库

- 4. 多进程

### 1. 提取每一页的电影列表,以及每个电影的详情页

In [3]:
print('你在干什么')

print('end=dfdfdsfsdfdfjsdfakjsdfioasdfi;j')





你在干什么？
你在干什么？

你在干什么
end=dfdfdsfsdfdfjsdfakjsdfioasdfi;j
